In [76]:
from sklearn.ensemble import IsolationForest
import pandas as pd
import numpy as np
import datetime
import warnings; warnings.filterwarnings('ignore')

In [77]:
# Load the sample spending data from CSV
daily_exp = pd.read_csv("Daily Expenditure.csv")
daily_exp.head(19)

,Date,Category,Subcategory,Description,Amount,Payment Method,Location,Merchant,Recurring,Income,Savings,Notes,Day
0,2024-06-01,Food,Groceries,Grocery shopping,50.0,Credit Card,New York,Walmart,Yes,150,100,Bought weekly supplies,Sunday
1,2024-06-01,Transportation,Public Transit,Bus ticket,2.5,Cash,New York,MTA,No,150,100,Daily commute,Monday
2,2024-06-01,Entertainment,Movies,Movie ticket,12.0,Debit Card,New York,AMC Theatres,No,150,100,Watched a new release,Tuesday
3,2024-06-02,Food,Dining Out,Restaurant lunch,25.0,Credit Card,New York,McDonald's,No,150,125,Lunch with colleagues,Wednesday
4,2024-06-02,Utilities,Electricity,Electricity bill,60.0,Debit Card,New York,ConEdison,Yes,150,90,Monthly bill payment,Thursday
5,2024-06-03,Healthcare,Medical,Doctor's visit,80.0,Credit Card,New York,City Hospital,No,200,120,Annual check-up,Friday
6,2024-12-01,Education,Books,Textbooks,40.0,Credit Card,New York,Barnes & Noble,No,200,160,Semester textbooks,Saturday
7,2024-02-22,Rent,Housing,Monthly rent,1200.0,Bank Transfer,New York,Landlord,Yes,1500,300,Rent for June,Saturday
8,2024-06-04,Transportation,Ride Share,Uber ride,20.0,Credit Card,New York,Uber,No,1500,300,Ride to airport,Sunday
9,2024-06-05,Food,Groceries,NaN,45.0,Cash,New York,Walmart,Yes,140,95,Weekly grocery run,Monday


In [78]:
#Step 1
#Preprocessing
#Null Check
daily_exp.isnull().sum()

Date               0
Category           0
Subcategory        0
Description        1
Amount             0
 Payment Method    0
Location           0
Merchant           0
Recurring          0
Income             0
Savings            0
Notes              0
Day                0
dtype: int64

In [79]:
daily_exp['Category'] = daily_exp['Category'].fillna(value="Food")
daily_exp['Description'] = daily_exp['Description'].fillna(value="Restaurant dinner")
daily_exp['Day'] = daily_exp['Day'].fillna(value="Saturday")
daily_exp.isnull().sum()

Date               0
Category           0
Subcategory        0
Description        0
Amount             0
 Payment Method    0
Location           0
Merchant           0
Recurring          0
Income             0
Savings            0
Notes              0
Day                0
dtype: int64

In [80]:
#Duplicate check
daily_exp.duplicated().sum()
daily_exp.drop_duplicates(inplace=True)
daily_exp.duplicated().sum()

0

In [81]:
#Incorrect format
def convert_date(date_str):
    return pd.to_datetime(date_str).date()
daily_exp['Date'] = daily_exp['Date'].apply(convert_date) 
daily_exp.head()

daily_exp['Day'] = daily_exp['Day'].replace('Satuday', 'Saturday')
daily_exp['Day'].unique()

array(['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
       'Saturday'], dtype=object)

In [82]:
#Step 2
#Data Analysis
for col in daily_exp.columns:
    print('Unique value of ' + col)
    print(daily_exp[col].unique())
    print('\n')

Unique value of Date
[datetime.date(2024, 6, 1) datetime.date(2024, 6, 2)
 datetime.date(2024, 6, 3) datetime.date(2024, 12, 1)
 datetime.date(2024, 2, 22) datetime.date(2024, 6, 4)
 datetime.date(2024, 6, 5) datetime.date(2024, 6, 6)
 datetime.date(2024, 6, 7) datetime.date(2024, 1, 22)]


Unique value of Category
['Food' 'Transportation' 'Entertainment' 'Utilities' 'Healthcare'
 'Education' 'Rent' 'Health' 'Clothing']


Unique value of Subcategory
['Groceries' 'Public Transit' 'Movies' 'Dining Out' 'Electricity'
 'Medical' 'Books' 'Housing' 'Ride Share' 'Concerts' 'Fitness' 'Apparel'
 'Fuel']


Unique value of Description
['Grocery shopping' 'Bus ticket' 'Movie ticket' 'Restaurant lunch'
 'Electricity bill' "Doctor's visit" 'Textbooks' 'Monthly rent'
 'Uber ride' 'Restaurant dinner' 'Concert ticket' 'Gym membership'
 'Shoes purchase' 'Gasoline']


Unique value of Amount
[  50.     2.5   12.    25.    60.    80.    40.  1200.    20.    45.
  100.    30.    70. ]


Unique value of  Pay

In [83]:
daily_exp['Date'].unique()
daily_exp['Date'] = daily_exp['Date'].replace(
                                               [datetime.date(2024, 6, 1), datetime.date(2024, 6, 2),datetime.date(2024, 6, 3),
                                               datetime.date(2024, 12, 1),datetime.date(2024, 2, 22), datetime.date(2024, 6, 4),
                                               datetime.date(2024, 6, 5), datetime.date(2024, 6, 6),datetime.date(2024, 6, 7),
                                               datetime.date(2024, 1, 22)],
                                               [1,2,3,4,5,6,7,8,9,10]
                                             )
daily_exp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 19
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             19 non-null     int64  
 1   Category         19 non-null     object 
 2   Subcategory      19 non-null     object 
 3   Description      19 non-null     object 
 4   Amount           19 non-null     float64
 5    Payment Method  19 non-null     object 
 6   Location         19 non-null     object 
 7   Merchant         19 non-null     object 
 8   Recurring        19 non-null     object 
 9   Income           19 non-null     int64  
 10  Savings          19 non-null     int64  
 11  Notes            19 non-null     object 
 12  Day              19 non-null     object 
dtypes: float64(1), int64(3), object(9)
memory usage: 2.1+ KB


In [84]:
daily_exp['Category'].unique()
daily_exp['Category'] = daily_exp['Category'].replace(
                                                 ['Food', 'Transportation', 'Entertainment', 'Utilities','Healthcare', 'Education', 'Rent',
                                                 'Health', 'Clothing'],
                                                 [1,2,3,4,5,6,7,8,9]
                                              )
daily_exp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 19
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             19 non-null     int64  
 1   Category         19 non-null     int64  
 2   Subcategory      19 non-null     object 
 3   Description      19 non-null     object 
 4   Amount           19 non-null     float64
 5    Payment Method  19 non-null     object 
 6   Location         19 non-null     object 
 7   Merchant         19 non-null     object 
 8   Recurring        19 non-null     object 
 9   Income           19 non-null     int64  
 10  Savings          19 non-null     int64  
 11  Notes            19 non-null     object 
 12  Day              19 non-null     object 
dtypes: float64(1), int64(4), object(8)
memory usage: 2.1+ KB


In [85]:
daily_exp['Subcategory'].unique()
daily_exp['Subcategory'] = daily_exp['Subcategory'].replace(
                                                ['Groceries', 'Public Transit', 'Movies', 'Dining Out','Electricity', 'Medical', 'Books',
                                                 'Housing', 'Ride Share','Concerts', 'Fitness', 'Apparel', 'Fuel'],
                                                 [1,2,3,4,5,6,7,8,9,10,11,12,13]
                                              )
daily_exp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 19
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             19 non-null     int64  
 1   Category         19 non-null     int64  
 2   Subcategory      19 non-null     int64  
 3   Description      19 non-null     object 
 4   Amount           19 non-null     float64
 5    Payment Method  19 non-null     object 
 6   Location         19 non-null     object 
 7   Merchant         19 non-null     object 
 8   Recurring        19 non-null     object 
 9   Income           19 non-null     int64  
 10  Savings          19 non-null     int64  
 11  Notes            19 non-null     object 
 12  Day              19 non-null     object 
dtypes: float64(1), int64(5), object(7)
memory usage: 2.1+ KB


In [86]:
daily_exp['Description'].unique()
daily_exp['Description'] = daily_exp['Description'].replace(
                                               ['Grocery shopping', 'Bus ticket', 'Movie ticket','Restaurant lunch', 'Electricity bill',
                                               "Doctor's visit",'Textbooks', 'Monthly rent', 'Uber ride', 'Restaurant dinner',
                                               'Concert ticket', 'Gym membership', 'Shoes purchase', 'Gasoline'],
                                                 [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
                                              )
daily_exp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 19
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             19 non-null     int64  
 1   Category         19 non-null     int64  
 2   Subcategory      19 non-null     int64  
 3   Description      19 non-null     int64  
 4   Amount           19 non-null     float64
 5    Payment Method  19 non-null     object 
 6   Location         19 non-null     object 
 7   Merchant         19 non-null     object 
 8   Recurring        19 non-null     object 
 9   Income           19 non-null     int64  
 10  Savings          19 non-null     int64  
 11  Notes            19 non-null     object 
 12  Day              19 non-null     object 
dtypes: float64(1), int64(6), object(6)
memory usage: 2.1+ KB


In [87]:
daily_exp[' Payment Method'].unique()
daily_exp[' Payment Method'] = daily_exp[' Payment Method'].replace(
                                               ['Credit Card', 'Cash', 'Debit Card', 'Bank Transfer'],
                                                 [1,2,3,4]
                                              )
daily_exp.info()
['Credit Card' 'Cash' 'Debit Card' 'Bank Transfer']

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 19
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             19 non-null     int64  
 1   Category         19 non-null     int64  
 2   Subcategory      19 non-null     int64  
 3   Description      19 non-null     int64  
 4   Amount           19 non-null     float64
 5    Payment Method  19 non-null     int64  
 6   Location         19 non-null     object 
 7   Merchant         19 non-null     object 
 8   Recurring        19 non-null     object 
 9   Income           19 non-null     int64  
 10  Savings          19 non-null     int64  
 11  Notes            19 non-null     object 
 12  Day              19 non-null     object 
dtypes: float64(1), int64(7), object(5)
memory usage: 2.1+ KB


['Credit CardCashDebit CardBank Transfer']

In [88]:
daily_exp['Location'].unique()
daily_exp['Location'] = daily_exp['Location'].replace(
                                              ['New York'],
                                                 [1]
                                              )
daily_exp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 19
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             19 non-null     int64  
 1   Category         19 non-null     int64  
 2   Subcategory      19 non-null     int64  
 3   Description      19 non-null     int64  
 4   Amount           19 non-null     float64
 5    Payment Method  19 non-null     int64  
 6   Location         19 non-null     int64  
 7   Merchant         19 non-null     object 
 8   Recurring        19 non-null     object 
 9   Income           19 non-null     int64  
 10  Savings          19 non-null     int64  
 11  Notes            19 non-null     object 
 12  Day              19 non-null     object 
dtypes: float64(1), int64(8), object(4)
memory usage: 2.1+ KB


In [89]:
daily_exp['Merchant'].unique()
daily_exp['Merchant'] = daily_exp['Merchant'].replace(
                                             ['Walmart', 'MTA', 'AMC Theatres', "McDonald's", 'ConEdison','City Hospital', 'Barnes & Noble',
                                              'Landlord', 'Uber','Ticketmaster', 'KFC', "Gold's Gym", 'Nike Store', 'Shell'],
                                                 [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
                                              )
daily_exp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 19
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             19 non-null     int64  
 1   Category         19 non-null     int64  
 2   Subcategory      19 non-null     int64  
 3   Description      19 non-null     int64  
 4   Amount           19 non-null     float64
 5    Payment Method  19 non-null     int64  
 6   Location         19 non-null     int64  
 7   Merchant         19 non-null     int64  
 8   Recurring        19 non-null     object 
 9   Income           19 non-null     int64  
 10  Savings          19 non-null     int64  
 11  Notes            19 non-null     object 
 12  Day              19 non-null     object 
dtypes: float64(1), int64(9), object(3)
memory usage: 2.1+ KB


In [90]:
daily_exp['Recurring'].unique()
daily_exp['Recurring'] = daily_exp['Recurring'].replace(
                                             ['Yes', 'No'],
                                                 [1,2]
                                              )
daily_exp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 19
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             19 non-null     int64  
 1   Category         19 non-null     int64  
 2   Subcategory      19 non-null     int64  
 3   Description      19 non-null     int64  
 4   Amount           19 non-null     float64
 5    Payment Method  19 non-null     int64  
 6   Location         19 non-null     int64  
 7   Merchant         19 non-null     int64  
 8   Recurring        19 non-null     int64  
 9   Income           19 non-null     int64  
 10  Savings          19 non-null     int64  
 11  Notes            19 non-null     object 
 12  Day              19 non-null     object 
dtypes: float64(1), int64(10), object(2)
memory usage: 2.1+ KB


In [91]:
daily_exp['Notes'].unique()
daily_exp['Notes'] = daily_exp['Notes'].replace(
                                             ['Bought weekly supplies', 'Daily commute', 'Watched a new release','Lunch with colleagues',
                                             'Monthly bill payment', 'Annual check-up','Semester textbooks', 'Rent for June', 'Ride to airport',
                                             'Weekly grocery run', 'Live music event', 'Dinner with friends','Monthly gym fee', 'New running shoes',
                                             'Filled up the tank'],
                                                 [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
                                              )
daily_exp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 19
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             19 non-null     int64  
 1   Category         19 non-null     int64  
 2   Subcategory      19 non-null     int64  
 3   Description      19 non-null     int64  
 4   Amount           19 non-null     float64
 5    Payment Method  19 non-null     int64  
 6   Location         19 non-null     int64  
 7   Merchant         19 non-null     int64  
 8   Recurring        19 non-null     int64  
 9   Income           19 non-null     int64  
 10  Savings          19 non-null     int64  
 11  Notes            19 non-null     int64  
 12  Day              19 non-null     object 
dtypes: float64(1), int64(11), object(1)
memory usage: 2.1+ KB


In [92]:
daily_exp['Day'].unique()
daily_exp['Day'] = daily_exp['Day'].replace(
                                ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday'],
                                [1,2,3,4,5,6,7]
                                )
daily_exp.info()
daily_exp.head(20)

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 19
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             19 non-null     int64  
 1   Category         19 non-null     int64  
 2   Subcategory      19 non-null     int64  
 3   Description      19 non-null     int64  
 4   Amount           19 non-null     float64
 5    Payment Method  19 non-null     int64  
 6   Location         19 non-null     int64  
 7   Merchant         19 non-null     int64  
 8   Recurring        19 non-null     int64  
 9   Income           19 non-null     int64  
 10  Savings          19 non-null     int64  
 11  Notes            19 non-null     int64  
 12  Day              19 non-null     int64  
dtypes: float64(1), int64(12)
memory usage: 2.1 KB


,Date,Category,Subcategory,Description,Amount,Payment Method,Location,Merchant,Recurring,Income,Savings,Notes,Day
0,1,1,1,1,50.0,1,1,1,1,150,100,1,1
1,1,2,2,2,2.5,2,1,2,2,150,100,2,2
2,1,3,3,3,12.0,3,1,3,2,150,100,3,3
3,2,1,4,4,25.0,1,1,4,2,150,125,4,4
4,2,4,5,5,60.0,3,1,5,1,150,90,5,5
5,3,5,6,6,80.0,1,1,6,2,200,120,6,6
6,4,6,7,7,40.0,1,1,7,2,200,160,7,7
7,5,7,8,8,1200.0,4,1,8,1,1500,300,8,7
8,6,2,9,9,20.0,1,1,9,2,1500,300,9,1
9,7,1,1,10,45.0,2,1,1,1,140,95,10,2


In [93]:
input_data = daily_exp

In [94]:
#Train the Isolation Forest model
#n_jobs(This parameter specifies the number of CPU cores to use for the computation.If n_jobs=-1, all available CPU cores will be used.n_jobs=13 means that the computation will use 13 CPU cores in parallel.Using multiple cores can improve performance by parallelizing the work,particularly beneficial for large datasets or complex models.)
#random_state(is like setting a fixed pattern for shuffling cards, ensuring you get the same results each time, making your experiments more reliable and comparable.)
clf = IsolationForest(n_jobs=13,contamination=0.1, random_state=42)
clf.fit(input_data)

IsolationForest(contamination=0.1, n_jobs=13, random_state=42)

In [95]:
#Step 3: Predict anomalies
y_pred = clf.predict(input_data)
y_pred

array([ 1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1])

In [96]:
# -1 indicates anomaly, 1 indicates normal
outliers = input_data[y_pred == -1]
outliers

,Date,Category,Subcategory,Description,Amount,Payment Method,Location,Merchant,Recurring,Income,Savings,Notes,Day
7,5,7,8,8,1200.0,4,1,8,1,1500,300,8,7
8,6,2,9,9,20.0,1,1,9,2,1500,300,9,1


In [97]:
print("Number of outliers detected:", len(outliers))

Number of outliers detected: 2
